In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *

In [2]:
path='data/ml-latest-small/'

In [3]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies = pd.read_csv(path+'movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
val_idxs = get_cv_idxs(len(ratings)) # cross-validation indexes, since we don't have a separate validation set.
wd=2e-4 # weight decay (for the optimizer we're using)
n_factors = 50 # dimensionality of the embedding matrix

In [6]:
a = T([[1., 2], [3, 4]])
b = T([[2., 2], [10, 10]])
a,b

(
  1  2
  3  4
 [torch.FloatTensor of size 2x2], 
   2   2
  10  10
 [torch.FloatTensor of size 2x2])

In [7]:
a*b


  2   4
 30  40
[torch.FloatTensor of size 2x2]

In [8]:
(a*b).sum(1)


  6
 70
[torch.FloatTensor of size 2]

In [9]:
class DotProduct(nn.Module):
    """pytorch module to calculate 1-d dot product"""
    def forward(self, u, m): return (u*m).sum(1)

In [10]:
model = DotProduct()

In [11]:
model(a, b)


  6
 70
[torch.FloatTensor of size 2]

Because the `userId`s and `movieId`s may not be contiguous, we get the unique ones of each, enumerate them, and store the enumeration index in a dict. We then replace the `userId` and `movieId` fields in the ratings dataframe with the new contiguous integers. 

In [12]:
u_uniq = ratings.userId.unique()
user2idx = {o:i for i,o in enumerate(u_uniq)}
ratings.userId = ratings.userId.apply(lambda x: user2idx[x])

m_uniq = ratings.movieId.unique()

movie2idx = {o:i for i,o in enumerate(m_uniq)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x])

n_users = int(ratings.userId.nunique())
n_movies = int(ratings.movieId.nunique())

In [13]:
n_users,n_movies

(671, 9066)

The actual embedding matrix is a variable, not a tensor. The difference between the two is that a variable can perform automatic differentiation. To pull the tensor out of a variable, you get its `data` attribute. In PyTorch, all of the tensor functions have a version with an underscore at the end, e.g. `uniform` has `uniform_` in
```
self.u.weight.data.uniform_(0,0.05)
```
which enables you to calculate values in place

In [14]:
class EmbeddingDot(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        self.u = nn.Embedding(n_users, n_factors)
        self.m = nn.Embedding(n_movies, n_factors)
        self.u.weight.data.uniform_(0,0.05) # weight initialization
        self.m.weight.data.uniform_(0,0.05)
    
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1]
        u, m = self.u(users), self.m(movies)
        return (u*m).sum(1)

In [15]:
x = ratings.drop(['rating', 'timestamp'], axis=1)
y = ratings['rating'].values.astype(np.float32)

In [16]:
data = ColumnarModelData.from_data_frame(path, val_idxs, x, y, ['userId', 'movieId'], 64)

In [17]:
wd = 1e-5
lr = 1e-1
model = EmbeddingDot(n_users, n_movies).cuda()
opt = optim.SGD(model.parameters(), lr, weight_decay=wd, momentum=0.9)

`fit` is a fastai library function that loops through each minibatch to fit a model. If you don't want to perform your own training loop but want to avoid as much of `fastai` as possible, this is the function to call directly. However, you won't have access to fastai tools such as SGDR, differential learning rates etc.

In [18]:
fit(model, data, 10, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

[ 0.       1.67627  1.63617]                                   
[ 1.       1.13087  1.30792]                                   
[ 2.       0.90835  1.22542]                                    
[ 3.       0.76516  1.20123]                                    
[ 4.       0.72462  1.17098]                                    
[ 5.       0.61049  1.16034]                                    
[ 6.       0.48634  1.16154]                                    
[ 7.      0.4008  1.1639]                                       
[ 8.       0.35235  1.17702]                                    
[ 9.       0.27996  1.19229]                                    



manual learning rate annealing

In [20]:
set_lrs(opt, 0.01)

In [21]:
fit(model, data, 10, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

[ 0.       0.105    1.22152]                                    
[ 1.       0.09901  1.22067]                                     
[ 2.       0.10646  1.22151]                                     
[ 3.       0.09281  1.22359]                                     
[ 4.       0.09455  1.22489]                                     
[ 5.       0.09407  1.22691]                                     
[ 6.       0.09106  1.22806]                                     
[ 7.       0.08558  1.2298 ]                                     
[ 8.       0.0834   1.23103]                                     
[ 9.       0.08504  1.23302]                                     



In [22]:
min_rating, max_rating = ratings.rating.min(), ratings.rating.max()
min_rating, max_rating

(0.5, 5.0)

In [23]:
def get_emb(ni, nf):
    """takes a number of inputs and a number of factors 
    (number of rows,columns in the embedding matrix), and returns an 
    initialized embedding matrix"""
    e = nn.Embedding(ni, nf)
    e.weight.data.uniform_(-0.01, 0.01)
    return e

In [24]:
class EmbeddingDotBias(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors), 
            (n_users, 1), (n_movies, 1)
        ]]
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1]
        um = (self.u(users) * self.m(movies)).sum(1)
        res = um + self.ub(users).squeeze() + self.mb(movies).squeeze()
        res = F.sigmoid(res) * (max_rating - min_rating) + min_rating
        return res

In [25]:
wd = 2e-4
lr = 1e-1
model = EmbeddingDotBias(n_users, n_movies).cuda()
opt = optim.SGD(model.parameters(), lr, weight_decay=wd, momentum=0.9)

In [27]:
fit(model, data, 10, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

[ 0.       0.75627  0.80457]                                    
[ 1.       0.7806   0.80371]                                    
[ 2.       0.7487   0.79998]                                    
[ 3.       0.72433  0.79548]                                    
[ 4.       0.67474  0.7894 ]                                    
[ 5.       0.64464  0.78774]                                    
[ 6.       0.62606  0.78304]                                    
[ 7.       0.62189  0.78066]                                    
[ 8.       0.57876  0.77787]                                    
[ 9.       0.56653  0.77536]                                    



In [23]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_users, n_movies, nh=10):
        super().__init__()
        (self.u, self.m) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors)
        ]]
        self.lin1 = nn.Linear(n_factors*2, nh)
        self.lin2 = nn.Linear(10, 1)
    
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1]
        x = torch.cat([self.u(users), self.m(movies)], dim=1)

        if self.training:
            x = F.dropout(torch.cat([self.u(users), self.m(movies)], dim=1), 0.75)
            x = F.dropout(F.relu(self.lin1(x)), 0.75)
        return F.sigmoid(self.lin2(x)) * (max_rating - min_rating + 1) + min_rating - 0.5

In [24]:
wd = 5e-4
lr = 1e-2
model = EmbeddingNet(n_users, n_movies, 10).cuda()
opt = optim.SGD(model.parameters(), lr, weight_decay=wd, momentum=0.9)

In [26]:
model.training = True
fit(model, data, 10, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))


  9%|▊         | 107/1251 [00:01<00:14, 77.27it/s, loss=0.817]


 17%|█▋        | 218/1251 [00:02<00:13, 76.35it/s, loss=0.809]


 26%|██▌       | 322/1251 [00:03<00:09, 96.68it/s, loss=0.806]


 36%|███▌      | 450/1251 [00:03<00:06, 118.82it/s, loss=0.825]


...

 74%|███████▎  | 920/1251 [00:05<00:02, 158.88it/s, loss=0.808]


 83%|████████▎ | 1033/1251 [00:06<00:01, 162.51it/s, loss=0.806]


 92%|█████████▏| 1152/1251 [00:06<00:00, 168.05it/s, loss=0.802]


100%|█████████▉| 1247/1251 [00:07<00:00, 172.23it/s, loss=0.81] 
                                                               


RuntimeError: size mismatch at /home/bmn/src/ai.learn/pytorch/pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:244

Adam = exponentially weighted moving average?

In [34]:
set_lrs(opt, 0.001)

In [36]:
fit(model, data, 10, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))


 18%|█▊        | 225/1251 [00:01<00:06, 151.25it/s, loss=0.585]


 37%|███▋      | 463/1251 [00:02<00:04, 185.17it/s, loss=0.582]


 54%|█████▍    | 678/1251 [00:03<00:02, 192.74it/s, loss=0.602]


 74%|███████▎  | 922/1251 [00:04<00:01, 202.37it/s, loss=0.595]


...

 28%|██▊       | 355/1251 [00:02<00:05, 174.57it/s, loss=0.58]


 47%|████▋     | 588/1251 [00:02<00:03, 197.17it/s, loss=0.605]


 67%|██████▋   | 839/1251 [00:03<00:01, 211.97it/s, loss=0.596]


 85%|████████▌ | 1067/1251 [00:04<00:00, 216.79it/s, loss=0.607]


 99%|█████████▉| 1240/1251 [00:05<00:00, 219.09it/s, loss=0.602]
                                                                [ 9.       0.60178  0.8018 ]

